## Extract currency from contextAllCurr (testSet). 
- if single curr found, take it as the Operationnal frequency
- if multiple currencies found, use scoringDict computed from the train set to associate a score to each curr. 
and retain the one associated to the highest score. 

In [1]:
# SYSTEM
from os import listdir
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *
from collections import Counter

#GENERAL
import numpy as np
import pandas as pd


### load labels and context dict.
the context dict is a dict. of dict. such as contextAllCurr = {'fileId': { ' curr' : Ngram } },
where Ngramm are part of speech centered around a currency. The contextAllCurr has been 
saved in a separate notebook (save_curencyContext_testSet.ipynb)

In [2]:
# get the solution
labels=pd.read_csv('../../data_0/greg_test_set/lab_greg_test_set.csv')
# get context dict
contextAllCurr=np.load('../../data_0/greg_test_set/contextAllCurr.npy').item()

extract only currencies found in the documents, store results in foundCurrType = {'fileId': [curr]}. 

In [3]:
foundCurrType=dict()

for f in labels.fileId:
    
    foundCurrType[f]=[]
    
    for currKey in contextAllCurr[f]:
        if contextAllCurr[f][currKey] != []:
            foundCurrType[f].append(currKey)

Divide the foundCurrType in three dicts:
 - single currency
 - multiple currency
 - void

In [4]:
voidCurr=dict()
singleCurr=dict()
multiCurr=dict()

for key in foundCurrType:
    if len(foundCurrType[key]) == 0:
        voidCurr[key]=foundCurrType[key]
    elif len(foundCurrType[key]) == 1:
        singleCurr[key]=foundCurrType[key]
    elif len(foundCurrType[key]) > 1:
        multiCurr[key]=foundCurrType[key]



## Extract currency

### Score multiple currencies

In [6]:
# load dict of weight
weight=np.load('../../data_0/greg_train_set/weight_forCurrencyScoring.npy').item()

tokenizer=RegexpTokenizer(r'\w+')

score=dict()

for f in multiCurr:
    score[f]=dict()
    for cur in multiCurr[f]:
        textTmp=[]
        score[f][cur]=1
        for val in contextAllCurr[f][cur]:
            textTmp.append(' '.join(val))
        
        textTmp=tokenizer.tokenize(str(textTmp).lower())
        for w in textTmp:
            if w in weight.keys():
                score[f][cur]=score[f][cur]*weight[w]

### build the dictionnary of extracted operationnal currency

In [7]:
opCurr=dict()

#multiple currency case: keep highest score currency
for f in score:
    opCurr[f]=max(score[f], key=score[f].get)

#individual currency case:    
for f in singleCurr:
    opCurr[f]=singleCurr[f][0]

#void currency case:
for f in voidCurr:
    opCurr[f]='000'

### save the extracted operationnal currency

In [8]:
np.save('../../data_0/greg_test_set/extractedOpCurr',opCurr)